In [118]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =["‘", 'chahie|', "\\", '/', 'tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        for i in data['Freq']:
            print(i)
        #for i in data['Pairs']:
         #   print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''prabhavita sandhi ke charom ora ki mansapeshiyom ko rogi ko sandhi ko gati dete hue ichchhapurvaka sikoda़na tatha shithila karana chahie, ise sthira vyayama kaha jata hai, aura hara ghante mem 2-3 minata taka kiya jata hai| jaise-jaise rogi ki sthiti mem sudhara hota hai, vyayama ko badha़ana chahie, para bahuta jaldi-jaldi nahim, nahim to bimari ka punaravartana ho sakata hai| aba dhire-dhire vyayama ko badha़ana chahie kintu atyasakti se bimari ka punaravartana ho sakata hai| rogi apane arama ke samaya ko kama kara sakata hai| khapachchi ko lambe samaya ke lie hataya ja sakata hai ya kevala rata ke samaya rakha ja sakata hai| dhire-dhire, sandhiyom ke vyayama se, gatishilata badha़ jaegi| rogi ko apane apa vyayama karana chahie aura kuchha vyayamom ko chikitsaka ki sahayata se kara sakata hai| vaha eka vakara ki sahayata se ya kisi ke sahare se chala sakata hai| vaha baisakhi se chala sakata hai| baisakhi pakshaghata se bachane ke lie kuhani baisakhi ki apeksha kankha ki baisakhi ka upayoga karem| baisakhi ke bada rogi chhada़i ke sahare se chala sakata hai| dhire-dhire rogi apani mansapeshiyom ke niyantrana ko badha़ana prarambha karata hai|
yaha saba usaki ayu, rogi ke prayatna tatha vaha isase shighra hi chhutakara pane ke lie kitana kritasankalpa hai, isa para nirbhara karega| jirna avastha mem vyayama atyavashyaka haim, nahim to vikriti nishchita hai| yadi sandhiyom ka uchita rupa se vyayama nahim kiya gaya, to ve muda़ne ki avastha mem bhi sthira ho jati haim, mansapeshiyam patali tatha vyartha ho jati haim aura uttejaka utpada sandhiyom ke charom ora jama ho jate haim| amavati sandhishotha mem kai anya vikritiyam hoti haim| yadi rogi narama bistara para sota raha hai, to graiva merudanda, thoda़i ke vakshasthala ke samipa pahunchane taka jukati hai| rogi aisa lagata hai jaise vaha eka gole mem guda़i-muda़i hokara leta hai| haddiyam asthisushira ho jati haim aura prayah thoda़e se sharirika prashikshana se tuta jati haim| ina rogiyom ko eka suniyojita aura paryavekshita vyayama upachara ki avashyakata hoti hai| dhire-dhire unaki gatishilata tatha mansapeshiyom ki kshamata badha़ti jaegi| vyayama se purva sandhiyom ko kuchha ushma upachara bhi die ja sakate haim| yaha mansapeshiyom ko shithila karane mem sahayata karata hai tatha rakta apurti ko badha़ata hai aura isilie yaha vyayama adhika prabhavi ho jata hai| yaha dhire-dhira sandhi ke kada़epana tatha sankuchana ko sudharata hai aura isamem chikitsaka tatha rogi donom ki ora se bahuta adhika dhairya tatha driढ़ta ki avashyakata hoti hai| prabhavita sandhi ke lie pahala vyayama chikitsaka dvara eka lambe satra mem sikhaya jana chahie| chikitsaka dvara vyayama ka kama se kama saptaha mem do bara nirikshana hona chahie tatha sudhara para dhyana diya jana chahie| vyayama tala sarvottama sahayaka haim, jaham rogi apani sandhiyom ko svatantratapurvaka sabhi dishaom mem chala sakata hai| vishesha sandhi vyayamom ke satha-satha samanya vyayama bhi diya jana chahie| mansapeshiyom ki kshamata ko badha़ane ke lie pratirodhaka vyayama die jane chahie jo sharira ki gatividhi mem tatha kama karate samaya sahara deti haim| yadi apaka vajana 70 kilo hai, to pani mem apake pairom ko kevala 7 kilo ko sahara dena hoga| yadi apa kamaja़ora pairom ke karana jaise adharangaghata mem ya kashtakara ghutanom ke karana jaise sandhishotha mem, khada़e nahim ho sakate, to sambhavatah apa santulana karane mem sakshama honge; pani apako aisa karane mem sahayata karega| pani sanstambhi mansapeshiyom ko shithila karata hai taki apa pani mem adhika asani se vyayama kara sakem| pani tanava kama karata hai aura yaha tanavapurna rogiyom ke lie mahattvapurna sahayata hai| yadi tala ke pani ka tapamana 37 digri sentigre़da hai to yaha apako behatara arama dega tatha darda aura pida़a ko kama karega| ghutanom ko sidhe rakhate hue pairom ko eka ke bada eka chalaem| kuchha minata chalane ke bada, rogi ke, pindaliyom ki mansapeshiyom mem tivra pida़a hoti hai aura vaha kuchha minatom ke arama se kama ho jati hai| yaha rakta vahikaom ki sankirnata ke karana hoti hai|
chalate samaya rakta apurti mem vriddhi honi hi chahie| jaba rogi chalata hai to rakta ki avashyakata badha़ jati hai aura yadi, rakta vahikaom ki sankirnata ke karana, rakta ki apurti mem vriddhi nahim hoti hai, to vaha eka araktatajanya sthiti mem pahuncha jati hai| ina rogiyom ko dhunrapana banda kara dena chahie, kyonki aise rogiyom mem sigareta pine ka hanikaraka prabhava hota hai| unhem pratidina eka ghante taka teja़i se chalane ke lie kaha jata hai| jaba pida़a ki punaravritti hoti hai to unhem kuchha minatom ke lie arama karane aura taba teja़i se chalana punah arambha karane ki anumati di jati hai| dhamanikakathinya abhilopa ke lakshanom vale rogiyo ko jo bistara ya kursi taka simita rahate haim, chikitsaka ya parivara ke kisi sadasya ki sahayata se kie jane vale kramika pratibandhatmaka gatividhiyom se vyayama karana chahie| vyayama karane ke lie ayu rukavata nahim hai| eka 67 sala ka vyakti savirama langada़epana ke satha kevala 5 blaka chala saka| unhem prarambha mem 1/2 se 1 kimi. taka chalana chahie aura jaise-jaise svasthya anumati deta hai vaise-vaise pha़asale ko dhire-dhire badha़ana chahie| ise sarala mukta hasta vyayamom ya si़dha़i vale vyayamom, mitara vale saikila ya treda mila ke satha sammilita kiya ja sakata hai|
unhem variyata se yadi duri kama ho to banisbata kara se jane ke chalana chahie| isa sudhara ke satha, unhem vyayama ke samaya hridaya gati mem giravata anubhava hoga| yaham taka ve bhi jinhem roga ke gambhira prakara haim, apane vyayama starom aura aksijana antargrahana ko bisa pratishata badha़a sakate haim| shvasana ke ‘punarprashikshana’ mem shvasana vyayama bahuta adhika sahayaka haim| hama dama, shvasanali-shotha aura vatasphiti ke lie inhim vyayamom ka prayoga karate haim| hamanem ina vyayamom mem se kai ke sabhi achchhe sarom ko sammilita karane ki koshisha ki hai| rogi ko chikitsaka para vishvasa hona chahie| unhem vyaktigata rupa se sikhaya jana chahie aura bada mem ve kisi samuha mem sammilita ho sakate haim| jaba vaha ina vyayamom ko samuha mem karata hai, to use dusarom se protsahana milega aura vaha adhika vishvasapurna anubhava karega tatha behatara prayasa karega| arama karana dama tatha vatasphiti ke rogi ke lie sabase mahattvapurna hai| ye rogi hamesha tanavapurna, ghabaraye hue tatha thake hue hote haim kyonki hara samaya ye shvasarodha se bachane ke lie lagatara prayasa karate rahate haim| yadi apa ina rogiyom ko dhyana se dekhenge to apa tanava ke kai lakshana paenge| unake hatha mem kampakampi ho sakati hai, jaba unaki ankhem banda ho to phada़ka sakati haim, aura unake vakshasthala ke kshetra mem alpa gatividhi ke paitarna ho sakate haim kyonki ve hara samaya apane shvasana ke bare mem sacheta rahate haim| ve upari vakshasthala, kandhem, gale tatha merudanda mem adhika gatividhi dikha sakate haim, visheshakara jaba unhem koi dekha raha ho| yaha tanava tatha ye vyartha ki gatividhiyam durbhagyavasha bahuta adhika urja khatma karate haim| yadi ve arama se haim, kevala taba hi ve adhika nipunata se aura kuchha sahi tarike se shvasana karana sikha sakate haim| rogi ko ina vyartha tividhiyom ke bare mem bataya jana chahie aura udara shvasana ki gudha़ta ko bhi sikhana chahie|
use yaha bataya jana chahie ki uchita shvasana kama kashtakara hoga tatha shvasahinata ke akramana ko bhi kama kara dega| ye nai pranaliyam alpavadhi pranaliyam nahim haim, aura sthayi rupa se prayoga kie jane chahie| akramana ke samaya use ina nai pranaliyom ki ora adhika dhyana dena chahie aura use isa nae jnana se labha hoga| ina rogiyom ko ghara mem anandadayaka aura anukula vatavarana pradana karana chahie| unhem eka nishchinta pravriti bhi apanani chahie| jaba rogi ko tanava mukta hona sikhaya jata hai, to eka nihshabdata ka vatavarana hona chahie| use usake sira aura ghutanom ko takiye se sahara dekara tatha banhom ko arama se vishrama karate hue arama se lita dena chahie| use tanavapurna aura tanavamukta mansapeshiyom ke bicha antara ke bare mem batana chahie| yaha taba taka jari rakhana chahie jaba taka ki vaha kaise unhem puri taraha se tanavamukta karana hai yaha nahim jana jata| ise banha, kandhe, gale, chehare, vakshasthala tatha udara ki mansapeshiyom mem karana chahie| mansapeshiyom ke ina tanavom ko mapa ja sakata hai, aura hamem unaki saphalata ke bare mem janakari ho jaegi| jaba rogi isa tanavamukta hone ki prakriya ko sikha leta hai, to vaha vyayama ke lie taiyara hai| vaha kandhe tatha upari vakshasthala ki chhoti mansapeshiyom ko tanane aura tanavamukta karane se prarambha karata hai| shvasana ke satha-satha isi uddeshya se yojanabaddha kie gae vividha layabaddha aura dolana vyayama karae jate haim| shvasana vyayamom ko subaha ya shama mem jaba peta khali hota hai, taba karem| khada़i avastha mansapeshi ki gatividhiyom ke lie svatantrata ka labha pradana karata hai| sidhe khada़e hom, banhe apake parshva mem hom, dhire se aura gahare shvasa chhoda़em, gale ko jukaem, apane kandhom ko age lae tatha peta ko sikoda़em| apa paenge ki apa aura adhika hava chhoda़ sakate haim| aba dhire se shvasa lena arambha karem, vakshasthala, peta ka vistara karem tatha gale ko sidha karem| thoda़a aura adhika shvasa lene ke lie, apane vakshasthala ko uncha uthaem, peta ko thoda़a khincha le tatha gale ko badha़aem| kuchha kshanom ke lie shvasa ko rokem aura dhire se shvasa chhoda़na prarambha karem tatha banhom aura eda़iyom ki gatividhi ko ulata dem| kuchha kshanom ke lie shvasa ko rokem aura ise punah prarambha karem| jaba pamva ke chapa samatala hom, to rogi ko chalate samaya pida़a ka anubhava hota hai|
ghutane bhi tirachhe sanketa karate haim, isaka artha hai ki pura pamva nitamba sandhi para ghuma gaya hai| adhikatara vikritiyam pahale chara mahinom mem utpanna hote haim, jaba bachcha palatane ke lie bahuta chhota hota hai| kisi bhi sthiti mem bachche ko rakha jaya, janma ke thika bada se hi, vaha apani aramadeha avastha janata hai| tani hui mansapeshiyam kamaja़ora ho jati haim tatha sikuda़i hui mansapeshiyam shaktishali ho jati haim| 
jaba pida़a tivra aura sandhi uttejita hoti hai to eka tivra avastha ko pahuncha jati hai| ina rogiyom ka atmavishvasa ke satha upachara hona chahie, rogi taka sandesha svatah hi pahuncha jaega tatha unhem janakari ho jaegi ki ve sahi hathom mem haim| eka patale suti gadde sahita eka lakada़i ka bencha paryapta hona chahie| padapata se bachane ke lie paira ko gaddidara gatte para rakha ja sakata hai| rata mem kevala eka patala takiya dena chahie| muda़i hui kohani ke satha, bahom ko sharira se juda़a hua rakhana chahie| 
bahom ki sthiti mem nirantara parivartana hote rahana chahie| sujana ghatati jati hai, aura marija ko darda se bahuta hi jalda arama mila jata hai| niche ki ora sira karake pharsha para leta jaem, hatha ko niche ki ora va hatheliyom ko pharsha para rakhem|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

5
3
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [116]:
from collections import OrderedDict
import numpy as np
import spacy
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        word_s =["‘", 'chahie|', "\\", '/', 'tha|', 'haim', 'haim|', '`', "'","’", '-', 'jaenge|', 'haim|', '"', 'hai)|', 'paenge|', '\n', '(', ')' ,'-', 'hai|','?', '!', ',', '|', '.', ';', ':', 'andar', 'ata', 'adi', 'apa', 'apana', 'apani', 'apani', 'apane', 'abhi', 'abhi',  'adi', 'apa', 'inhim', 'inhem', 'inhom', 'itayadi', 'ityadi', 'ina', 'inaka', 'inhim', 'inhem', 'inhom', 'isa', 'isaka', 'isaki', 'isaki', 'isake', 'isamem', 'isi', 'isi', 'ise', 'unhim', 'unhem', 'unhom', 'una', 'unaka', 'unaki', 'unaki', 'unake', 'unako', 'unhim', 'unhem', 'unhom', 'usa', 'usake', 'usi', 'usi', 'use', 'eka', 'evam', 'esa', 'ese', 'aise', 'ora', 'aura', 'kai', 'kai', 'kara', 'karata', 'karate', 'karana', 'karane', 'karem', 'kahate', 'kaha', 'ka', 'kaphi', 'kai', 'ki', 'kinhem', 'kinhom', 'kitana', 'kinhem', 'kinhom', 'kiya', 'kira', 'kisa', 'kisi', 'kisi', 'kise', 'ki', 'kuchha', 'kula', 'ke', 'ko', 'koi', 'koi', 'kona', 'konasa', 'kauna', 'kaunasa', 'gaya', 'ghara', 'jaba', 'jaham', 'jaham', 'ja', 'jinhem', 'jinhom', 'jitana', 'jidhara', 'jina', 'jinhem', 'jinhom', 'jisa', 'jise', 'jidhara', 'jesa', 'jese', 'jaisa', 'jaise', 'jo', 'taka', 'taba', 'taraha', 'tinhem', 'tinhom', 'tina', 'tinhem', 'tinhom', 'tisa', 'tise', 'to', 'tha', 'thi', 'thi', 'the', 'dabara', 'davara', 'diya', 'dusara', 'dusare', 'dusare', 'do', 'dvara', 'na', 'nahim', 'nahim', 'na', 'niche', 'nihayata', 'niche', 'ne', 'para', 'pahale', 'pura', 'pura', 'pe', 'phira', 'bani', 'bani', 'bahi', 'bahi', 'bahuta', 'bada', 'bala', 'bilakula', 'bhi', 'bhitara', 'bhi', 'bhitara', 'magara', 'mano', 'me', 'mem', 'yadi', 'yaha', 'yaham', 'yaham', 'yahi', 'yahi', 'ya', 'yiha', 'ye', 'rakhem', 'ravasa', 'raha', 'rahe', 'vasa', 'lie', 'liye', 'lekina', 'va', 'vageraha', 'varaga', 'varga', 'vaha', 'vaham', 'vaham', 'vahim', 'vahim', 'vale', 'vuha', 've', 'va', 'airaha', 'sanga', 'sakata', 'sakate', 'sabase', 'sabhi', 'sabhi', 'satha', 'sabuta', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone', 'sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone''sabha', 'sara', 'se', 'so', 'hi', 'hi', 'hua', 'hua', 'hui', 'hui', 'hue', 'he', 'hem', 'hai', 'haim', 'ho', 'hota', 'hoti', 'hoti', 'hote', 'hona', 'hone']
       
        for word in word_s:
           lexeme = nlp.vocab[word]
           lexeme.is_stop = True 
            
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                #if token.pos_ in candidate_pos and token.is_stop is False:
                if token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
            
       # print(sentences)    
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        token_pairs_freq = []
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])    
                    token_pairs.append(pair)
        #for word in token_pairs:
         #   print(word)
        unique_values = []
        unique_values = pd.Series(token_pairs).unique()
        #for i in unique_values:
            #print(i)
            
        arr = pd.DataFrame([])
       
        ls = []
        for i in unique_values:
            count = 0
            for j in token_pairs:
                if i == j:
                    count = count + 1
                else:
                    continue
            #print(i, count)
            lm = []
            lm = [i,count]
            ls.append(lm)
            #arr = arr.append(pd.DataFrame({'Pairs': pd.Series(data = [i]), 'Freq': pd.Series(data = [count])}))
        
        #for word in ls:
         #   print(word)
            
        data = pd.DataFrame(ls,columns=['Pairs','Freq'])
        data = data.sort_values('Freq', ascending = False)
        #for i in data['Freq']:
         #   print(i)
        for i in data['Pairs']:
            print(i)
        print(data)
         
      
        
        #print(token_pairs_freq)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break
        
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight
        
        
        
text = '''prabhavita sandhi ke charom ora ki mansapeshiyom ko rogi ko sandhi ko gati dete hue ichchhapurvaka sikoda़na tatha shithila karana chahie, ise sthira vyayama kaha jata hai, aura hara ghante mem 2-3 minata taka kiya jata hai| jaise-jaise rogi ki sthiti mem sudhara hota hai, vyayama ko badha़ana chahie, para bahuta jaldi-jaldi nahim, nahim to bimari ka punaravartana ho sakata hai| aba dhire-dhire vyayama ko badha़ana chahie kintu atyasakti se bimari ka punaravartana ho sakata hai| rogi apane arama ke samaya ko kama kara sakata hai| khapachchi ko lambe samaya ke lie hataya ja sakata hai ya kevala rata ke samaya rakha ja sakata hai| dhire-dhire, sandhiyom ke vyayama se, gatishilata badha़ jaegi| rogi ko apane apa vyayama karana chahie aura kuchha vyayamom ko chikitsaka ki sahayata se kara sakata hai| vaha eka vakara ki sahayata se ya kisi ke sahare se chala sakata hai| vaha baisakhi se chala sakata hai| baisakhi pakshaghata se bachane ke lie kuhani baisakhi ki apeksha kankha ki baisakhi ka upayoga karem| baisakhi ke bada rogi chhada़i ke sahare se chala sakata hai| dhire-dhire rogi apani mansapeshiyom ke niyantrana ko badha़ana prarambha karata hai|
yaha saba usaki ayu, rogi ke prayatna tatha vaha isase shighra hi chhutakara pane ke lie kitana kritasankalpa hai, isa para nirbhara karega| jirna avastha mem vyayama atyavashyaka haim, nahim to vikriti nishchita hai| yadi sandhiyom ka uchita rupa se vyayama nahim kiya gaya, to ve muda़ne ki avastha mem bhi sthira ho jati haim, mansapeshiyam patali tatha vyartha ho jati haim aura uttejaka utpada sandhiyom ke charom ora jama ho jate haim| amavati sandhishotha mem kai anya vikritiyam hoti haim| yadi rogi narama bistara para sota raha hai, to graiva merudanda, thoda़i ke vakshasthala ke samipa pahunchane taka jukati hai| rogi aisa lagata hai jaise vaha eka gole mem guda़i-muda़i hokara leta hai| haddiyam asthisushira ho jati haim aura prayah thoda़e se sharirika prashikshana se tuta jati haim| ina rogiyom ko eka suniyojita aura paryavekshita vyayama upachara ki avashyakata hoti hai| dhire-dhire unaki gatishilata tatha mansapeshiyom ki kshamata badha़ti jaegi| vyayama se purva sandhiyom ko kuchha ushma upachara bhi die ja sakate haim| yaha mansapeshiyom ko shithila karane mem sahayata karata hai tatha rakta apurti ko badha़ata hai aura isilie yaha vyayama adhika prabhavi ho jata hai| yaha dhire-dhira sandhi ke kada़epana tatha sankuchana ko sudharata hai aura isamem chikitsaka tatha rogi donom ki ora se bahuta adhika dhairya tatha driढ़ta ki avashyakata hoti hai| prabhavita sandhi ke lie pahala vyayama chikitsaka dvara eka lambe satra mem sikhaya jana chahie| chikitsaka dvara vyayama ka kama se kama saptaha mem do bara nirikshana hona chahie tatha sudhara para dhyana diya jana chahie| vyayama tala sarvottama sahayaka haim, jaham rogi apani sandhiyom ko svatantratapurvaka sabhi dishaom mem chala sakata hai| vishesha sandhi vyayamom ke satha-satha samanya vyayama bhi diya jana chahie| mansapeshiyom ki kshamata ko badha़ane ke lie pratirodhaka vyayama die jane chahie jo sharira ki gatividhi mem tatha kama karate samaya sahara deti haim| yadi apaka vajana 70 kilo hai, to pani mem apake pairom ko kevala 7 kilo ko sahara dena hoga| yadi apa kamaja़ora pairom ke karana jaise adharangaghata mem ya kashtakara ghutanom ke karana jaise sandhishotha mem, khada़e nahim ho sakate, to sambhavatah apa santulana karane mem sakshama honge; pani apako aisa karane mem sahayata karega| pani sanstambhi mansapeshiyom ko shithila karata hai taki apa pani mem adhika asani se vyayama kara sakem| pani tanava kama karata hai aura yaha tanavapurna rogiyom ke lie mahattvapurna sahayata hai| yadi tala ke pani ka tapamana 37 digri sentigre़da hai to yaha apako behatara arama dega tatha darda aura pida़a ko kama karega| ghutanom ko sidhe rakhate hue pairom ko eka ke bada eka chalaem| kuchha minata chalane ke bada, rogi ke, pindaliyom ki mansapeshiyom mem tivra pida़a hoti hai aura vaha kuchha minatom ke arama se kama ho jati hai| yaha rakta vahikaom ki sankirnata ke karana hoti hai|
chalate samaya rakta apurti mem vriddhi honi hi chahie| jaba rogi chalata hai to rakta ki avashyakata badha़ jati hai aura yadi, rakta vahikaom ki sankirnata ke karana, rakta ki apurti mem vriddhi nahim hoti hai, to vaha eka araktatajanya sthiti mem pahuncha jati hai| ina rogiyom ko dhunrapana banda kara dena chahie, kyonki aise rogiyom mem sigareta pine ka hanikaraka prabhava hota hai| unhem pratidina eka ghante taka teja़i se chalane ke lie kaha jata hai| jaba pida़a ki punaravritti hoti hai to unhem kuchha minatom ke lie arama karane aura taba teja़i se chalana punah arambha karane ki anumati di jati hai| dhamanikakathinya abhilopa ke lakshanom vale rogiyo ko jo bistara ya kursi taka simita rahate haim, chikitsaka ya parivara ke kisi sadasya ki sahayata se kie jane vale kramika pratibandhatmaka gatividhiyom se vyayama karana chahie| vyayama karane ke lie ayu rukavata nahim hai| eka 67 sala ka vyakti savirama langada़epana ke satha kevala 5 blaka chala saka| unhem prarambha mem 1/2 se 1 kimi. taka chalana chahie aura jaise-jaise svasthya anumati deta hai vaise-vaise pha़asale ko dhire-dhire badha़ana chahie| ise sarala mukta hasta vyayamom ya si़dha़i vale vyayamom, mitara vale saikila ya treda mila ke satha sammilita kiya ja sakata hai|
unhem variyata se yadi duri kama ho to banisbata kara se jane ke chalana chahie| isa sudhara ke satha, unhem vyayama ke samaya hridaya gati mem giravata anubhava hoga| yaham taka ve bhi jinhem roga ke gambhira prakara haim, apane vyayama starom aura aksijana antargrahana ko bisa pratishata badha़a sakate haim| shvasana ke ‘punarprashikshana’ mem shvasana vyayama bahuta adhika sahayaka haim| hama dama, shvasanali-shotha aura vatasphiti ke lie inhim vyayamom ka prayoga karate haim| hamanem ina vyayamom mem se kai ke sabhi achchhe sarom ko sammilita karane ki koshisha ki hai| rogi ko chikitsaka para vishvasa hona chahie| unhem vyaktigata rupa se sikhaya jana chahie aura bada mem ve kisi samuha mem sammilita ho sakate haim| jaba vaha ina vyayamom ko samuha mem karata hai, to use dusarom se protsahana milega aura vaha adhika vishvasapurna anubhava karega tatha behatara prayasa karega| arama karana dama tatha vatasphiti ke rogi ke lie sabase mahattvapurna hai| ye rogi hamesha tanavapurna, ghabaraye hue tatha thake hue hote haim kyonki hara samaya ye shvasarodha se bachane ke lie lagatara prayasa karate rahate haim| yadi apa ina rogiyom ko dhyana se dekhenge to apa tanava ke kai lakshana paenge| unake hatha mem kampakampi ho sakati hai, jaba unaki ankhem banda ho to phada़ka sakati haim, aura unake vakshasthala ke kshetra mem alpa gatividhi ke paitarna ho sakate haim kyonki ve hara samaya apane shvasana ke bare mem sacheta rahate haim| ve upari vakshasthala, kandhem, gale tatha merudanda mem adhika gatividhi dikha sakate haim, visheshakara jaba unhem koi dekha raha ho| yaha tanava tatha ye vyartha ki gatividhiyam durbhagyavasha bahuta adhika urja khatma karate haim| yadi ve arama se haim, kevala taba hi ve adhika nipunata se aura kuchha sahi tarike se shvasana karana sikha sakate haim| rogi ko ina vyartha tividhiyom ke bare mem bataya jana chahie aura udara shvasana ki gudha़ta ko bhi sikhana chahie|
use yaha bataya jana chahie ki uchita shvasana kama kashtakara hoga tatha shvasahinata ke akramana ko bhi kama kara dega| ye nai pranaliyam alpavadhi pranaliyam nahim haim, aura sthayi rupa se prayoga kie jane chahie| akramana ke samaya use ina nai pranaliyom ki ora adhika dhyana dena chahie aura use isa nae jnana se labha hoga| ina rogiyom ko ghara mem anandadayaka aura anukula vatavarana pradana karana chahie| unhem eka nishchinta pravriti bhi apanani chahie| jaba rogi ko tanava mukta hona sikhaya jata hai, to eka nihshabdata ka vatavarana hona chahie| use usake sira aura ghutanom ko takiye se sahara dekara tatha banhom ko arama se vishrama karate hue arama se lita dena chahie| use tanavapurna aura tanavamukta mansapeshiyom ke bicha antara ke bare mem batana chahie| yaha taba taka jari rakhana chahie jaba taka ki vaha kaise unhem puri taraha se tanavamukta karana hai yaha nahim jana jata| ise banha, kandhe, gale, chehare, vakshasthala tatha udara ki mansapeshiyom mem karana chahie| mansapeshiyom ke ina tanavom ko mapa ja sakata hai, aura hamem unaki saphalata ke bare mem janakari ho jaegi| jaba rogi isa tanavamukta hone ki prakriya ko sikha leta hai, to vaha vyayama ke lie taiyara hai| vaha kandhe tatha upari vakshasthala ki chhoti mansapeshiyom ko tanane aura tanavamukta karane se prarambha karata hai| shvasana ke satha-satha isi uddeshya se yojanabaddha kie gae vividha layabaddha aura dolana vyayama karae jate haim| shvasana vyayamom ko subaha ya shama mem jaba peta khali hota hai, taba karem| khada़i avastha mansapeshi ki gatividhiyom ke lie svatantrata ka labha pradana karata hai| sidhe khada़e hom, banhe apake parshva mem hom, dhire se aura gahare shvasa chhoda़em, gale ko jukaem, apane kandhom ko age lae tatha peta ko sikoda़em| apa paenge ki apa aura adhika hava chhoda़ sakate haim| aba dhire se shvasa lena arambha karem, vakshasthala, peta ka vistara karem tatha gale ko sidha karem| thoda़a aura adhika shvasa lene ke lie, apane vakshasthala ko uncha uthaem, peta ko thoda़a khincha le tatha gale ko badha़aem| kuchha kshanom ke lie shvasa ko rokem aura dhire se shvasa chhoda़na prarambha karem tatha banhom aura eda़iyom ki gatividhi ko ulata dem| kuchha kshanom ke lie shvasa ko rokem aura ise punah prarambha karem| jaba pamva ke chapa samatala hom, to rogi ko chalate samaya pida़a ka anubhava hota hai|
ghutane bhi tirachhe sanketa karate haim, isaka artha hai ki pura pamva nitamba sandhi para ghuma gaya hai| adhikatara vikritiyam pahale chara mahinom mem utpanna hote haim, jaba bachcha palatane ke lie bahuta chhota hota hai| kisi bhi sthiti mem bachche ko rakha jaya, janma ke thika bada se hi, vaha apani aramadeha avastha janata hai| tani hui mansapeshiyam kamaja़ora ho jati haim tatha sikuda़i hui mansapeshiyam shaktishali ho jati haim| 
jaba pida़a tivra aura sandhi uttejita hoti hai to eka tivra avastha ko pahuncha jati hai| ina rogiyom ka atmavishvasa ke satha upachara hona chahie, rogi taka sandesha svatah hi pahuncha jaega tatha unhem janakari ho jaegi ki ve sahi hathom mem haim| eka patale suti gadde sahita eka lakada़i ka bencha paryapta hona chahie| padapata se bachane ke lie paira ko gaddidara gatte para rakha ja sakata hai| rata mem kevala eka patala takiya dena chahie| muda़i hui kohani ke satha, bahom ko sharira se juda़a hua rakhana chahie| 
bahom ki sthiti mem nirantara parivartana hote rahana chahie| sujana ghatati jati hai, aura marija ko darda se bahuta hi jalda arama mila jata hai| niche ki ora sira karake pharsha para leta jaem, hatha ko niche ki ora va hatheliyom ko pharsha para rakhem|
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=2, lower=False)
tr4w.get_keywords(25)

('dhire', 'dhire')
('jati', 'rogiyom')
('jana', 'chahie')
('prabhavita', 'sandhi')
('badha़ana', 'chahie')
('jati', 'rakta')
('rakta', 'vahikaom')
('kshanom', 'shvasa')
('chalate', 'samaya')
('upari', 'vakshasthala')
('bataya', 'jana')
('tatha', 'gale')
('hara', 'samaya')
('kie', 'jane')
('rakta', 'apurti')
('aba', 'dhire')
('jaegi|', 'rogi')
('sikhaya', 'jana')
('dena', 'tanavapurna')
('rupa', 'prayoga')
('shvasana', 'gudha़ta')
('gudha़ta', 'sikhana')
('sikhana', 'bataya')
('chahie', 'uchita')
('uchita', 'shvasana')
('shvasana', 'kama')
('jana', 'jata|')
('kashtakara', 'hoga')
('hoga', 'tatha')
('tatha', 'shvasahinata')
('pranaliyam', 'alpavadhi')
('kaise', 'puri')
('rakhana', 'chahie')
('alpavadhi', 'pranaliyam')
('udara', 'shvasana')
('chahie', 'udara')
('jata|', 'banha')
('vakshasthala', 'tatha')
('urja', 'khatma')
('khatma', 'arama')
('adhika', 'nipunata')
('nipunata', 'sahi')
('sahi', 'tarike')
('tarike', 'shvasana')
('chehare', 'vakshasthala')
('bare', 'bataya')
('gale', 'cheha

('lambe', 'satra')
('jaegi|', 'vyayama')
('jana', 'chikitsaka')
('chikitsaka', 'vyayama')
('kama', 'kama')
('kama', 'saptaha')
('bara', 'nirikshana')
('nirikshana', 'chahie')
('chahie', 'tatha')
('chikitsaka', 'tatha')
('sudharata', 'chikitsaka')
('sankuchana', 'sudharata')
('tatha', 'sankuchana')
('vyayama', 'purva')
('purva', 'sandhiyom')
('ushma', 'upachara')
('upachara', 'die')
('mansapeshiyom', 'shithila')
('shithila', 'sahayata')
('sahayata', 'tatha')
('tatha', 'rakta')
('apurti', 'badha़ata')
('badha़ata', 'isilie')
('isilie', 'vyayama')
('adhika', 'prabhavi')
('prabhavi', 'jata')
('dhire', 'dhira')
('dhira', 'sandhi')
('patali', 'tatha')
('jati', 'mansapeshiyam')
('sudhara', 'dhyana')
('kintu', 'atyasakti')
('sudhara', 'vyayama')
('vyayama', 'badha़ana')
('chahie', 'jaldi')
('jaldi', 'jaldi')
('jaldi', 'bimari')
('dhire', 'vyayama')
('chahie', 'kintu')
('bimari', 'punaravartana')
('sandhiyom', 'vyayama')
('rogi', 'arama')
('samaya', 'kama')
('khapachchi', 'lambe')
('lambe', 'sa

[634 rows x 2 columns]
tatha - 9.954698220144053
vyayama - 7.99871403028339
rogi - 6.672678703941962
chahie - 5.41502385338427
jati - 3.9911590117444287
adhika - 3.2902535546794476
samaya - 3.0749793125855622
mansapeshiyom - 3.0456120491005905
arama - 2.83549338520276
sandhi - 2.7241671083546084
shvasana - 2.7172443685943684
dhire - 2.566152271324755
kama - 2.5184930814941233
vakshasthala - 2.5179273288439954
vyayamom - 2.27380768999519
gale - 2.223360489818823
karega| - 2.1742735945767198
dena - 2.0984622519841265
jana - 2.0792971786315535
baisakhi - 1.9775
pani - 1.9523767615486363
leta - 1.9280087031024529
chikitsaka - 1.8853396289281705
sahayata - 1.8528424531872445
tanava - 1.8485308602662767
rakta - 1.8348034561888729
jata - 1.7681354207135458
